1. Introducción

    Predicción de Tipos de Cambio por Divisa

    En este notebook construiremos un modelo para anticipar la evolución del tipo de cambio (por ejemplo: EUR/USD, EUR/GBP, etc.). Esta estimación será esencial para decidir el mejor momento para comprar divisas y maximizar la rentabilidad.

2. Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Modelos posibles
from prophet import Prophet
from statsmodels.tsa.arima.model import ARIMA


3. Carga y transformación de tasas

    Carga y preparación del dataset de tasas de cambio

    Aquí trabajaremos con las tasas históricas EUR->USD, EUR->GBP, etc. Convertimos fechas y seleccionamos la divisa a modelar.


In [ ]:
df_tasas = pd.read_csv("data/tipos_cambio.csv")
df_tasas["fecha"] = pd.to_datetime(df_tasas["fecha"])

# Filtrar por una divisa concreta
divisa_base = "USD"
serie = df_tasas[df_tasas["divisa"] == divisa_base][["fecha", "tasa_cambio"]].set_index("fecha")


4. Visualización inicial

    Evolución histórica del tipo de cambio

    Visualizamos el comportamiento de la tasa EUR → {divisa} para evaluar tendencia, estacionalidad y volatilidad.


In [ ]:
serie.plot(figsize=(10, 4), title=f"Tasa de cambio histórica: EUR → {divisa_base}")
plt.ylabel("Tasa de cambio")
plt.show()


5. Ajuste de modelo (Prophet o ARIMA)

    Modelado del tipo de cambio

    Evaluamos dos opciones:
    - Prophet: útil si hay componentes estacionales o tendencias complejas
    - ARIMA: cuando la serie es más lineal y estacionaria tras diferenciación


In [ ]:
# Preparar datos para Prophet
df_model = serie.reset_index().rename(columns={"fecha": "ds", "tasa_cambio": "y"})

# Modelo Prophet
modelo = Prophet(daily_seasonality=True)
modelo.fit(df_model)
futuro = modelo.make_future_dataframe(periods=30)
prediccion = modelo.predict(futuro)


6. Visualización de predicción

    Predicción futura del tipo de cambio

    Mostramos la predicción con intervalos de confianza.


In [ ]:
modelo.plot(prediccion)
plt.title(f"Predicción de tipo de cambio EUR → {divisa_base}")
plt.show()


7. Evaluación

    Evaluación del modelo

    Si hacemos split temporal, comparamos predicción contra valores reales.
    Métricas:
    - MAE (error absoluto medio)
    - RMSE (raíz del error cuadrático medio)


In [ ]:
# Ejemplo si dividieras entre entrenamiento/test
# y calculas errores como:
print("MAE:", mean_absolute_error(y_real, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_real, y_pred)))


8. Guardado del modelo y exportación

    Podemos guardar el modelo ajustado y los resultados de la predicción para integrarlos en el sistema de optimización.